In [8]:
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix

foldseek_result = "../foldseek_pdb_subset_1000/results.tsv"
# headers = ["query","target","fident","alnlen","mismatch","gapopen","qstart","qend","tstart","tend","evalue","bits"]

In [9]:
df = pd.read_csv(foldseek_result, sep="\t")
for column in ["query", "target"]:
    df[column] = df[column].str.replace('_', ':')
df.head(2)

,query,target,fident,evalue,bits,lddt,alntmscore,rmsd
0,P42212,P42212,1.000,6.364000e-55,2299,1.0000,1.0000,0.006248
1,P42212,SS28T:SF112Y:SN144D:SK160R,0.983,6.613000e-53,2201,0.9924,0.9929,0.048610


In [5]:
# Get unique sequence IDs and create mapping to integers
all_seqs = sorted(list(set(df['query'].unique()) | set(df['target'].unique())))
seq_to_idx = {seq: idx for idx, seq in enumerate(all_seqs)}

# Convert sequences to integer indices
row_idx = np.array([seq_to_idx[seq] for seq in df['query']])
col_idx = np.array([seq_to_idx[seq] for seq in df['target']])

# Create sparse matrix in COO format
n = len(all_seqs)
sparse_matrix = coo_matrix(
    (df["fident"], (row_idx, col_idx)),
    shape=(n, n)
)

# Convert to dense matrix
dense_matrix = sparse_matrix.toarray()

# Convert to DataFrame with proper labels
matrix = pd.DataFrame(
    dense_matrix,
    index=all_seqs,
    columns=all_seqs
)

output_file = "../processed_data/structure_fident_matrix.csv"
matrix.to_csv(output_file, sep=',')

In [6]:
count_zeros = np.count_nonzero(matrix == 0)
count_zeros

0

In [7]:
output_file = "../processed_data/structure_similarity_matrix.csv"
matrix.to_csv(output_file, sep=',')